***
# 3.1 First Order Syntax for Bird 5
### JX, 03/15/2022
***
    Bird5:
    LS110131.10.9.2015GeoPool.aiff
    LS110136.10.10.2015GeoGreens.aiff
    LS110143.10.12.2015GeoGreens.aiff
    LS110146A.10.13.2015GeoGreens.aiff
    
***

In [1]:
indv = 'Bird5'

In [2]:
## Load up Note Dataframe

import pandas as pd
from pathlib2 import Path

from butcherbird.utils.paths import DATA_DIR

SAVE_LOC = 'interim/fine_labels/butcherbird_note_df_'+indv+'_fine.pickle'
SAVE_LOC = DATA_DIR / SAVE_LOC

note_df = pd.read_pickle(SAVE_LOC)

## Make a copy of recursively generated labels
#note_df['lasso_labels'] = note_df['recur_labels_id']

note_df

,phrase_nb,phrase_strt,phrase_end,phrase_len,note_cnt,note_nb,note_strt,note_end,note_len,indv,...,key,audio,rate,spectrogram,umap,hdbscan_labels,recur_labels,recur_labels_id,lasso_labels,seq_id
0,0,513.780000,517.061923,3.281923,15,0,513.780000,513.905214,0.125214,Bird5,...,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -7.356902671940375e-05, -0.000162745769...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4.8774133, 15.248685]",16,16|1|0|nan|nan|nan|nan|nan|nan,87,55,0
1,0,513.780000,517.061923,3.281923,15,1,513.917747,513.980936,0.063188,Bird5,...,LS110143.10.12.2015GeoGreens_dn,"[0.0, 0.0014728324725323874, 0.002769917049663...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[6.9320498, 10.134182]",13,13|1|0|1.0|0.0|1.0|0.0|nan|nan,45,58,0
2,0,513.780000,517.061923,3.281923,15,2,514.170000,514.480698,0.310698,Bird5,...,LS110143.10.12.2015GeoGreens_dn,"[0.0, 4.4531726447127805e-05, 4.62385657049665...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[8.438871, 21.151648]",5,5|2|0|0.0|nan|nan|nan|nan|nan,143,8,0
3,0,513.780000,517.061923,3.281923,15,3,514.620783,515.310000,0.689217,Bird5,...,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -3.2631556531601904e-06, -7.44037823939...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[19.637224, -5.1326833]",7,7|0|1|0.0|nan|nan|nan|nan|nan,151,31,0
4,0,513.780000,517.061923,3.281923,15,4,515.355217,515.660180,0.304964,Bird5,...,LS110143.10.12.2015GeoGreens_dn,"[-0.0, -2.958346783314775e-05, -7.119856074752...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[9.055997, -1.52321]",12,12|1|0|nan|nan|nan|nan|nan|nan,38,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17325,488,3955.500491,3957.771268,2.270777,16,11,3956.827269,3956.984287,0.157018,Bird5,...,LS110146A.10.13.2015GeoGreens_dn,"[-0.0, -0.0003734800289541283, -0.000744419409...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5.5191817, -9.9033375]",8,8|1|0|nan|nan|nan|nan|nan|nan,161,9,1770
17326,488,3955.500491,3957.771268,2.270777,16,12,3957.129954,3957.413722,0.283767,Bird5,...,LS110146A.10.13.2015GeoGreens_dn,"[-0.0, -5.999877314464557e-06, -1.294964036885...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[17.48298, -3.0183713]",15,15|1|0|nan|nan|nan|nan|nan|nan,81,34,1770
17327,488,3955.500491,3957.771268,2.270777,16,13,3957.530000,3957.576257,0.046257,Bird5,...,LS110146A.10.13.2015GeoGreens_dn,"[-0.0, -0.00020799144552458053, -0.00033384171...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4.470899, -3.3807]",4,4|1|1|0.0|1.0|0.0|nan|nan|nan,138,45,1770
17328,488,3955.500491,3957.771268,2.270777,16,14,3957.585716,3957.644361,0.058645,Bird5,...,LS110146A.10.13.2015GeoGreens_dn,"[0.0, 0.0007728977791414965, 0.000127507273682...",44100,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4.4345427, -3.3523924]",4,4|1|1|0.0|1.0|0.0|nan|nan|nan,138,45,1770


## Feed parameters into analysis package

In [3]:
onsets = note_df['note_strt']
syntactic_units = note_df['lasso_labels']
expected_interval_range = [0, 1]
dataset_description = 'Songbird: Pied Butcherbirds'

## Find Consistent Transitions

In [4]:
from butcherbird import sequential

In [5]:
import numpy as np

In [6]:
trans_mat = sequential.construct_seq_matrix(note_df, label = 'lasso_labels', sort = False)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]


In [7]:
## bootstrap null distributions with 100 samples under 0-order syntax assumption

In [8]:
p_container = []

## sample 100 times
for i in np.arange(0, 100):
    
    ## bootstrap one zero-order sequence
    bootstrapped_df = note_df.sample(n = len(note_df), replace = True).reset_index()
    
    ## find the transition matrix of the sequence
    bootstrapped_mat = sequential.construct_seq_matrix(bootstrapped_df, label = 'lasso_labels', sort = False)
    
    ## turn the matrix into long form
    bootstrapped_long = sequential.long_cvtr(bootstrapped_mat, songType = 'Bootstrapped')
    
    ## return bootstrapped distribution of all transition probabilities
    p_container.append(bootstrapped_long.p.values)
    
p_container

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 27 28 29 30 31 32 33 34 35 3

[array([0.02083333, 0.03125   , 0.046875  , ..., 0.        , 0.        ,
        0.01149425]),
 array([0.        , 0.03553299, 0.01522843, ..., 0.        , 0.00952381,
        0.00952381]),
 array([0.01421801, 0.01895735, 0.04739336, ..., 0.03947368, 0.        ,
        0.        ]),
 array([0.02232143, 0.03125   , 0.04017857, ..., 0.        , 0.        ,
        0.        ]),
 array([0.01      , 0.03      , 0.06      , ..., 0.        , 0.01162791,
        0.        ]),
 array([0.01363636, 0.02272727, 0.02727273, ..., 0.        , 0.01219512,
        0.        ]),
 array([0.01895735, 0.04265403, 0.06635071, ..., 0.        , 0.        ,
        0.        ]),
 array([0.01388889, 0.03240741, 0.05092593, ..., 0.01190476, 0.        ,
        0.01190476]),
 array([0.01408451, 0.02816901, 0.03286385, ..., 0.        , 0.        ,
        0.        ]),
 array([0.01244813, 0.06224066, 0.02904564, ..., 0.        , 0.        ,
        0.02150538]),
 array([0.00431034, 0.03448276, 0.05603448, ..., 0

In [9]:
## swap axes so that each element in p_container is a null sampling distribution of 
## find how likely that specific transition happpens with just 0 order syntax

In [10]:
p = np.swapaxes(p_container, 0, 1)

In [11]:
p

array([[0.02083333, 0.        , 0.01421801, ..., 0.00505051, 0.        ,
        0.        ],
       [0.03125   , 0.03553299, 0.01895735, ..., 0.03030303, 0.04265403,
        0.03414634],
       [0.046875  , 0.01522843, 0.04739336, ..., 0.05050505, 0.03791469,
        0.02926829],
       ...,
       [0.        , 0.        , 0.03947368, ..., 0.01086957, 0.        ,
        0.01204819],
       [0.        , 0.00952381, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01149425, 0.00952381, 0.        , ..., 0.        , 0.01234568,
        0.        ]])

In [12]:
import numpy as np
import scipy.stats

## a standard calculation for confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [13]:
higherconfints = []

for null_transitions in p:
    higherconfints.append(mean_confidence_interval(null_transitions, confidence = 0.95)[2])
    
higherconfints

[0.013456289637194106,
 0.0342214707517251,
 0.046156399283817914,
 0.02072480953532964,
 0.007258306119181111,
 0.0052698295812279245,
 0.006908891645500048,
 0.005094806826783396,
 0.018550527809549223,
 0.03965950285845736,
 0.026912240782802847,
 0.020398959036543526,
 0.0069558805934739456,
 0.01898339245416809,
 0.019363695572880917,
 0.03243547574230383,
 0.002322033502918509,
 0.005738127994336916,
 0.002661981568890047,
 0.006352685299662738,
 0.001338358396936992,
 0.0008506431159372263,
 0.006788624620428548,
 0.0005615072139167144,
 0.0,
 0.0052957970446536745,
 0.0,
 0.01595810464741437,
 0.01634296246660904,
 0.053738041346671594,
 0.05237012108158748,
 0.006367825234386865,
 0.016071692306612848,
 0.02345476921100884,
 0.05075836095534404,
 0.005632907061585785,
 0.005925459556180722,
 0.0029086779730563804,
 0.004579260406057529,
 0.004778739434824156,
 0.00954895008122539,
 0.008842498255694083,
 0.01053304974992038,
 0.015735950108105804,
 0.039560055729277774,
 0.021

In [14]:
trans_long = sequential.long_cvtr(trans_mat, songType = 'True')

In [15]:
trans_long

,songType,noteType,markov,p
0,True,0,0,0.004717
1,True,0,1,0.000000
2,True,0,2,0.000000
3,True,0,3,0.004717
4,True,0,4,0.000000
...,...,...,...,...
5324,True,72,68,0.000000
5325,True,72,69,0.010989
5326,True,72,70,0.043956
5327,True,72,71,0.000000


In [16]:
trans_long['higherconfint'] = higherconfints

In [17]:
trans_long

,songType,noteType,markov,p,higherconfint
0,True,0,0,0.004717,0.013456
1,True,0,1,0.000000,0.034221
2,True,0,2,0.000000,0.046156
3,True,0,3,0.004717,0.020725
4,True,0,4,0.000000,0.007258
...,...,...,...,...,...
5324,True,72,68,0.000000,0.007424
5325,True,72,69,0.010989,0.011129
5326,True,72,70,0.043956,0.008694
5327,True,72,71,0.000000,0.005781


In [18]:
filtered_trans_long = trans_long[trans_long.p > trans_long.higherconfint]

In [19]:
filtered_trans_long

,songType,noteType,markov,p,higherconfint
6,True,0,6,0.037736,0.006909
9,True,0,9,0.080189,0.039660
27,True,0,27,0.179245,0.015958
36,True,0,36,0.009434,0.005925
44,True,0,44,0.108491,0.039560
...,...,...,...,...,...
5320,True,72,64,0.010989,0.009298
5322,True,72,66,0.032967,0.007081
5323,True,72,67,0.032967,0.013078
5326,True,72,70,0.043956,0.008694


In [20]:
ftl = filtered_trans_long

## Rhythm Dyadic Analysis

In [21]:
## load rhythm analyzer
import butcherbird.rhythm as rhythm

/home/AD/j8xing/syntax_rhythm_pbb/butcherbird/rhythm.py:151: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [22]:
dyadic = rhythm.construct_rhythm_df(
    onsets = onsets, 
    syntactic_units = syntactic_units, 
    expected_interval_range = expected_interval_range)

In [23]:
dyadic

,intervals1,intervals2,label1,label2,label3,s_interval,l_interval,cycle_dur,ratio_roeske,ratio_custom,cycle_rank
6572,0.036773,0.003309,14,48,25,0.003309,0.036773,0.040083,0.917441,0.089989,1
1010,0.034970,0.005312,30,48,2,0.005312,0.034970,0.040282,0.868121,0.151913,2
72,0.039629,0.001440,14,48,11,0.001440,0.039629,0.041068,0.964946,0.036327,3
12132,0.045232,0.011358,45,45,28,0.011358,0.045232,0.056590,0.799288,0.251113,4
6555,0.022714,0.034077,6,6,50,0.022714,0.034077,0.056791,0.399952,0.666534,5
...,...,...,...,...,...,...,...,...,...,...,...
12402,0.411473,0.858550,51,32,41,0.411473,0.858550,1.270023,0.323989,0.479265,13789
3,0.734433,0.537019,31,48,14,0.537019,0.734433,1.271452,0.577633,0.731202,13790
8044,0.460000,0.820000,51,32,41,0.460000,0.820000,1.280000,0.359375,0.560976,13791
8144,0.440000,0.875788,51,32,41,0.440000,0.875788,1.315788,0.334400,0.502405,13792


In [24]:
ftl

,songType,noteType,markov,p,higherconfint
6,True,0,6,0.037736,0.006909
9,True,0,9,0.080189,0.039660
27,True,0,27,0.179245,0.015958
36,True,0,36,0.009434,0.005925
44,True,0,44,0.108491,0.039560
...,...,...,...,...,...
5320,True,72,64,0.010989,0.009298
5322,True,72,66,0.032967,0.007081
5323,True,72,67,0.032967,0.013078
5326,True,72,70,0.043956,0.008694


In [25]:
## verify rhythm counts above 0.01 of total rhythm count (8013 * 0.01)

In [26]:
verified = []

for index, transition in ftl.iterrows():
    note = transition.noteType
    dest = transition.markov
    
    ## find if dyadic contains enough entries for this particular combination
    first_filter = dyadic[dyadic.label1 == note]
    second_filter = first_filter[first_filter.label2 == dest]
    
    if len(second_filter) > 0.005*len(dyadic):
        verified.append(index)

In [27]:
vr = ftl.filter(items = verified, axis = 0)

In [28]:
unique, counts = np.unique(vr.noteType, return_counts = True)

In [29]:
eligible_notes = []
for noteType, count in zip(unique, counts):
    if count > 1:
        eligible_notes.append(noteType)

In [30]:
eligible_transitions = []

for n in eligible_notes:
    eligible_transitions.append(vr[vr.noteType == n])
    
eligible_transitions = pd.concat(eligible_transitions)

In [31]:
eligible_transitions

,songType,noteType,markov,p,higherconfint
102,True,1,29,0.512963,0.056068
103,True,1,30,0.198148,0.049961
122,True,1,49,0.129630,0.015848
757,True,10,27,0.423163,0.015858
774,True,10,44,0.213808,0.040519
1125,True,15,30,0.185039,0.049923
1143,True,15,48,0.501969,0.028420
2118,True,29,1,0.096878,0.032112
2128,True,29,11,0.207750,0.018778
2129,True,29,12,0.081808,0.007457


In [32]:
import scipy.stats

def bif_rhythm_ks(vd_bif, dyadic, mode = 'ratio_roeske'):
    '''
    Calculate KS test for each bifurcation point
    '''
    
    output = []
    
    # 1. Extract Bifurcation Info from vd_bif
    ### a. Find all pre-bifurcation notes
    pre_bif = np.unique(vd_bif['noteType'])
    
    # 2. For each bifurcation, conduct a Kolmogorov-Smirnov test
    ### a. for each bifurcation
    for note in pre_bif:
        ### a. Query its exact slice
        bif_df = vd_bif[vd_bif['noteType'] == note]
        ### b. Query its bifurcation destinations (assumes only 2)
        t1 = bif_df['markov'].values[0]
        t2 = bif_df['markov'].values[1]
        ### c. Filter dyadic by pre_bif
        note_df = dyadic[dyadic['label1'] == note]
        ### d. Filter note_df by bif destinations
        t1_df = note_df[note_df['label2'] == t1]
        t2_df = note_df[note_df['label2'] == t2]
        ### e. compute ks
        try:
            ks = scipy.stats.ks_2samp(t1_df[mode], t2_df[mode])
        except:
            ks = [None, None]
            print('ks not computed')
        ### e. feeds out pandas
        output.append(
            pd.DataFrame(
                {
                    'pre_bif': [note],
                    'bif_des': [(t1, t2)],
                    'bif_prof': [
                        (
                            bif_df['p'].values[0],
                            bif_df['p'].values[1]
                        )
                    ],
                    'rhy_cnts': [
                        (
                            len(t1_df),
                            len(t2_df)
                        )
                    ],
                    'ks_statistic': [ks[0]],
                    'ks_p': [ks[1]]
                }
            )
        )
        
        print('Finished querying bifurcation with pre_bif ', note)
    return pd.concat(output)

In [33]:
entangle = bif_rhythm_ks(eligible_transitions, dyadic)

Finished querying bifurcation with pre_bif  1
Finished querying bifurcation with pre_bif  10
Finished querying bifurcation with pre_bif  15
Finished querying bifurcation with pre_bif  29
Finished querying bifurcation with pre_bif  30
Finished querying bifurcation with pre_bif  34
Finished querying bifurcation with pre_bif  44
Finished querying bifurcation with pre_bif  47
Finished querying bifurcation with pre_bif  50
Finished querying bifurcation with pre_bif  58


In [34]:
entangle

,pre_bif,bif_des,bif_prof,rhy_cnts,ks_statistic,ks_p
0,1,"(29, 30)","(0.512962962962963, 0.19814814814814816)","(277, 107)",0.314012,3.021636e-07
0,10,"(27, 44)","(0.42316258351893093, 0.21380846325167038)","(190, 96)",0.764583,1.149089e-37
0,15,"(30, 48)","(0.18503937007874016, 0.5019685039370079)","(94, 255)",0.782103,1.107478e-42
0,29,"(1, 11)","(0.0968783638320775, 0.2077502691065662)","(89, 189)",0.325783,3.241363e-06
0,30,"(1, 13)","(0.11016949152542373, 0.09806295399515738)","(91, 81)",0.312305,3.243450e-04
0,34,"(44, 57)","(0.26889714993804215, 0.16480793060718713)","(217, 133)",0.995392,7.012887e-98
0,44,"(1, 2)","(0.1362962962962963, 0.2740740740740741)","(88, 185)",0.255221,6.455095e-04
0,47,"(44, 45)","(0.5105740181268882, 0.24169184290030213)","(169, 71)",0.936328,1.477330e-49
0,50,"(14, 51)","(0.1766304347826087, 0.3804347826086957)","(130, 222)",1.000000,9.491043e-100
0,58,"(8, 50)","(0.41379310344827586, 0.3630831643002028)","(202, 173)",0.351428,9.203423e-11


In [35]:
entangle['ks_p'] < 0.001

0    True
0    True
0    True
0    True
0    True
0    True
0    True
0    True
0    True
0    True
Name: ks_p, dtype: bool

In [36]:
## bonferroni correct

In [37]:
import statsmodels.stats.multitest

In [38]:
entangle['bonferroni_p'] = statsmodels.stats.multitest.multipletests(entangle['ks_p'].values, alpha = 0.05, method = 'bonferroni')[1]

In [39]:
entangle

,pre_bif,bif_des,bif_prof,rhy_cnts,ks_statistic,ks_p,bonferroni_p
0,1,"(29, 30)","(0.512962962962963, 0.19814814814814816)","(277, 107)",0.314012,3.021636e-07,3.021636e-06
0,10,"(27, 44)","(0.42316258351893093, 0.21380846325167038)","(190, 96)",0.764583,1.149089e-37,1.149089e-36
0,15,"(30, 48)","(0.18503937007874016, 0.5019685039370079)","(94, 255)",0.782103,1.107478e-42,1.107478e-41
0,29,"(1, 11)","(0.0968783638320775, 0.2077502691065662)","(89, 189)",0.325783,3.241363e-06,3.241363e-05
0,30,"(1, 13)","(0.11016949152542373, 0.09806295399515738)","(91, 81)",0.312305,3.243450e-04,3.243450e-03
0,34,"(44, 57)","(0.26889714993804215, 0.16480793060718713)","(217, 133)",0.995392,7.012887e-98,7.012887e-97
0,44,"(1, 2)","(0.1362962962962963, 0.2740740740740741)","(88, 185)",0.255221,6.455095e-04,6.455095e-03
0,47,"(44, 45)","(0.5105740181268882, 0.24169184290030213)","(169, 71)",0.936328,1.477330e-49,1.477330e-48
0,50,"(14, 51)","(0.1766304347826087, 0.3804347826086957)","(130, 222)",1.000000,9.491043e-100,9.491043e-99
0,58,"(8, 50)","(0.41379310344827586, 0.3630831643002028)","(202, 173)",0.351428,9.203423e-11,9.203423e-10


In [40]:
entangle['bonferroni_p'] < 0.001

0     True
0     True
0     True
0     True
0    False
0     True
0    False
0     True
0     True
0     True
Name: bonferroni_p, dtype: bool

In [41]:
## do a shuffle
dyadic_shuffle = dyadic.copy(deep = True)
np.random.shuffle(dyadic_shuffle['ratio_roeske'].values)
entangle_shuffled = bif_rhythm_ks(eligible_transitions, dyadic_shuffle)

Finished querying bifurcation with pre_bif  1
Finished querying bifurcation with pre_bif  10
Finished querying bifurcation with pre_bif  15
Finished querying bifurcation with pre_bif  29
Finished querying bifurcation with pre_bif  30
Finished querying bifurcation with pre_bif  34
Finished querying bifurcation with pre_bif  44
Finished querying bifurcation with pre_bif  47
Finished querying bifurcation with pre_bif  50
Finished querying bifurcation with pre_bif  58


In [42]:
entangle_shuffled

,pre_bif,bif_des,bif_prof,rhy_cnts,ks_statistic,ks_p
0,1,"(29, 30)","(0.512962962962963, 0.19814814814814816)","(277, 107)",0.076791,0.716715
0,10,"(27, 44)","(0.42316258351893093, 0.21380846325167038)","(190, 96)",0.092544,0.599786
0,15,"(30, 48)","(0.18503937007874016, 0.5019685039370079)","(94, 255)",0.074259,0.809437
0,29,"(1, 11)","(0.0968783638320775, 0.2077502691065662)","(89, 189)",0.064562,0.943688
0,30,"(1, 13)","(0.11016949152542373, 0.09806295399515738)","(91, 81)",0.209470,0.038221
0,34,"(44, 57)","(0.26889714993804215, 0.16480793060718713)","(217, 133)",0.083192,0.582217
0,44,"(1, 2)","(0.1362962962962963, 0.2740740740740741)","(88, 185)",0.069226,0.912380
0,47,"(44, 45)","(0.5105740181268882, 0.24169184290030213)","(169, 71)",0.094425,0.719781
0,50,"(14, 51)","(0.1766304347826087, 0.3804347826086957)","(130, 222)",0.075953,0.694738
0,58,"(8, 50)","(0.41379310344827586, 0.3630831643002028)","(202, 173)",0.067018,0.763269


# Compute Global Statistic

The concept of the global statistic here is to compare the rhythm ratios for each first-order syntax and to compare between-group variation / within-group variation. 

1. Find each first-order transition -> z-score -> compile all first-order transition -> MSE
2. Compare with zero-order transition -> z-score -> compile all first-order transition -> MSE
3. Is first-order transition variance is lower than zero-order transition variance?

## Let's find all the first-order transitions first

In [43]:
import scipy.stats as stats

In [44]:
def syntactic_rhythmic_modulation_metric (dyadic):
    
    ## compute global metric
    
    first_order_rhythms_centered = []
    first_order_group_n = 0

    ## for every possible label
    for label in np.unique(dyadic['label1']):

        ## find a sub dataframe containing only designated zero-order label
        zero_order = dyadic[dyadic['label1'] == label]

        ## for every possible transition of that label
        for transition in np.unique(zero_order['label2']):

            ## find its rhythms
            first_order = zero_order[zero_order['label2'] == transition]
            first_order_rhythms = first_order['ratio_roeske'].values
            first_order_rhythms_centered.extend(first_order_rhythms - np.mean(first_order_rhythms))

            ## add 1 to group_n
            first_order_group_n+=1
    
    # find MSE
    MSE_first = sum(np.square(first_order_rhythms_centered)) / len(dyadic)
    MSE_baseline = sum(np.square(dyadic['ratio_roeske'].values - np.mean(dyadic['ratio_roeske']))) / len(dyadic)
    
    return 1 - MSE_first / MSE_baseline

## empirical 

In [45]:
syntactic_rhythmic_modulation_metric(dyadic)

0.9590165013970394

## Find Baseline using bootstrap

In [46]:
from tqdm.autonotebook import tqdm

In [47]:
dyadic_shuffle = dyadic.copy(deep = True)

SDoMSE = []

for sample in tqdm(np.arange(0, 100)):
    
    ## shuffle dyadic
    np.random.shuffle(dyadic_shuffle['ratio_roeske'].values)
    
    SDoMSE.append(syntactic_rhythmic_modulation_metric(dyadic_shuffle))

  0%|          | 0/100 [00:00<?, ?it/s]

In [48]:
mean_confidence_interval(SDoMSE, confidence=0.95)

(0.06802750080426818, 0.0674951101215161, 0.06855989148702027)

In [49]:
syntactic_rhythmic_modulation_metric(dyadic)

0.9590165013970394

## try using anova

In [50]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [51]:
dyadic = dyadic.astype({'label1': str, 'label2': str})

In [52]:
model = ols('ratio_roeske ~ label1', data = dyadic).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
label1,519.333851,68.0,517.527628,0.0
Residual,202.527914,13724.0,NaN,NaN


In [53]:
dyadic['transition'] = dyadic['label1'] + '+' + dyadic['label2']

In [54]:
dyadic

,intervals1,intervals2,label1,label2,label3,s_interval,l_interval,cycle_dur,ratio_roeske,ratio_custom,cycle_rank,transition
6572,0.036773,0.003309,14,48,25,0.003309,0.036773,0.040083,0.917441,0.089989,1,14+48
1010,0.034970,0.005312,30,48,2,0.005312,0.034970,0.040282,0.868121,0.151913,2,30+48
72,0.039629,0.001440,14,48,11,0.001440,0.039629,0.041068,0.964946,0.036327,3,14+48
12132,0.045232,0.011358,45,45,28,0.011358,0.045232,0.056590,0.799288,0.251113,4,45+45
6555,0.022714,0.034077,6,6,50,0.022714,0.034077,0.056791,0.399952,0.666534,5,6+6
...,...,...,...,...,...,...,...,...,...,...,...,...
12402,0.411473,0.858550,51,32,41,0.411473,0.858550,1.270023,0.323989,0.479265,13789,51+32
3,0.734433,0.537019,31,48,14,0.537019,0.734433,1.271452,0.577633,0.731202,13790,31+48
8044,0.460000,0.820000,51,32,41,0.460000,0.820000,1.280000,0.359375,0.560976,13791,51+32
8144,0.440000,0.875788,51,32,41,0.440000,0.875788,1.315788,0.334400,0.502405,13792,51+32


In [55]:
model = ols('ratio_roeske ~ transition', data = dyadic).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
transition,692.277344,940.0,319.933645,0.0
Residual,29.584421,12852.0,NaN,NaN
